In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-154101")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-154101
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-154101


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_name = "hyperdrive-aml"

try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
except Exception:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-08-09T17:54:28.233000+00:00', 'errors': None, 'creationTime': '2021-08-09T17:54:05.897603+00:00', 'modifiedTime': '2021-08-09T17:54:31.468259+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [16]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": choice(0.01, 0.1, 1),
    "--max_iter": choice(20, 40, 60, 100, 150, 200)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

script_folder = "./training"
os.makedirs(script_folder, exist_ok=True)
shutil.copy("./train.py", script_folder)

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=script_folder, entry_script="train.py", compute_target=compute_target, vm_size="Standard_D2_V2", vm_priority="lowpriority")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, primary_metric_name="accuracy", primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=4, policy=policy, estimator=est)

In [17]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_ff096ffa-cd69-47e5-9063-f79f05a7226c
Web View: https://ml.azure.com/runs/HD_ff096ffa-cd69-47e5-9063-f79f05a7226c?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-154101/workspaces/quick-starts-ws-154101&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-08-09T18:15:19.690768][API][INFO]Experiment created<END>\n""<START>[2021-08-09T18:15:20.171066][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-08-09T18:15:20.404385][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_ff096ffa-cd69-47e5-9063-f79f05a7226c
Web View: https://ml.azure.com/runs/HD_ff096ffa-cd69-47e5-9063-f79f05a7226c?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-154101/workspaces/quick-starts-ws-154101&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_ff096ffa-cd69-47e5-9063-f79f05a7226c',
 'target': 'hyperdrive-aml',
 'status': 'Completed',
 'startTimeUtc': '2021-08-09T18:15:19.396928Z',
 'endTimeUtc': '2021-08-09T18:17:21.696876Z',
 'properties': {'primary_metric_config': '{"name": "accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '8f5c720b-e355-4596-995c-b1a80fe22e95',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.32.0',
  'score': '0.9128983308042489',
  'best_child_run_id': 'HD_ff096ffa-cd69-47e5-9063-f79f05a7226c_3',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg154101.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_ff096ffa-cd69-47e5-9063-f79f05a7226c/azureml-logs/hyperdrive.txt?sv=2019-07-07&sr=

In [21]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ####
hyperdrive_best_run = hyperdrive_run.get_best_run_by_primary_metric()

print("best run file names :", hyperdrive_best_run.get_file_names())

model = hyperdrive_best_run.register_model(model_name='hyperdrive_log_reg_best_run', 
model_path='outputs/model.joblib', tags={"Method" : "HyperDrive"},
properties={"accuracy" : hyperdrive_best_run.get_metrics()["accuracy"]})

best run file names : ['azureml-logs/55_azureml-execution-tvmps_ad23c0a7cb262b7845ad01198afa5171c2156f10679f7295c41ee47a5c4d7670_d.txt', 'azureml-logs/65_job_prep-tvmps_ad23c0a7cb262b7845ad01198afa5171c2156f10679f7295c41ee47a5c4d7670_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_ad23c0a7cb262b7845ad01198afa5171c2156f10679f7295c41ee47a5c4d7670_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/105_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


In [25]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [27]:
import pandas as pd


from train import clean_data
from sklearn.model_selection import train_test_split
from azureml.core import Dataset

# Use the clean_data function to clean your data.
x, y = clean_data(ds)


all_data = pd.concat([x,y], axis=1)

train_data, test_data = train_test_split(all_data, test_size=0.2, random_state=49)

train_data.to_csv('training/training_df.csv')

ds = ws.get_default_datastore()
ds.upload(src_dir='training', target_path='training_data')

train_ds = Dataset.Tabular.from_delimited_files(path=[(ds, ('training_data/training_df.csv'))])

Uploading an estimated of 4 files
Target already exists. Skipping upload for training_data/.amlignore
Target already exists. Skipping upload for training_data/.amlignore.amltmp
Target already exists. Skipping upload for training_data/train.py
Target already exists. Skipping upload for training_data/training_df.csv
Uploaded 0 files


In [37]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=60,
    task='classification',
    primary_metric='accuracy',
    training_data=train_ds,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=compute_target,
    enable_early_stopping = True,
    enable_onnx_compatible_models = True)

In [38]:
# Submit your automl run

### YOUR CODE HERE ###

remote = exp.submit(config=automl_config, show_output=True)
RunDetails(remote).show()

Submitting remote run.
No run_configuration provided, running on hyperdrive-aml with default configuration
Running on remote compute: hyperdrive-aml


Experiment,Id,Type,Status,Details Page,Docs Page
aml-quickstarts-153935,AutoML_6bddcc75-9974-4960-94fe-880bf8da0647,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-------------------------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [47]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ##

best_class_automl, best_fit_model = remote.get_output()
print(best_fit_model)

joblib.dump(best_fit_model, 'outputs/best_fit_automl_model.joblib')
best_class_automl.register_model(model_name = 'best_automl.pkl', model_path = '/outputs/')

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=0.7291666666666667, reg_lambda=2.3958333333333335, subsample=0.8, tree_method='auto'))], verbose=False)), ('7', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('logisticregression', LogisticRegression(C=1.7575106248547894, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='multinomial', n_jobs=1, penalty='l2', random_state=None, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))], verbose=False)), ('2', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('ext

Model(workspace=Workspace.create(name='quick-starts-ws-154101', subscription_id='3e42d11f-d64d-4173-af9b-12ecaa1030b3', resource_group='aml-quickstarts-154101'), name=best_automl.pkl, id=best_automl.pkl:1, version=1, tags={}, properties={})

In [48]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

